In [2]:
from utils.utils import *
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import metrics
import time
import sklearn
import numpy as np

/home/wjunneng/Python/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wjunneng/Python/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wjunneng/Python/anaconda3/envs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/w

In [4]:
trainset = sklearn.datasets.load_files(container_path = 'dataset', encoding = 'UTF-8')
trainset.data, trainset.target = separate_dataset(trainset,1.0)
print (trainset.target_names)
print (len(trainset.data))
print (len(trainset.target))

['negative', 'positive']
10662
10662


In [5]:
ONEHOT = np.zeros((len(trainset.data),len(trainset.target_names)))
ONEHOT[np.arange(len(trainset.data)),trainset.target] = 1.0
train_X, test_X, train_Y, test_Y, train_onehot, test_onehot = train_test_split(trainset.data, 
                                                                               trainset.target, 
                                                                               ONEHOT, test_size = 0.2)

In [6]:
data, count, dictionary, rev_dictionary, vocabulary_size = build_dataset(' '.join(trainset.data).split())
print('vocab from size: %d'%(vocabulary_size))
print('Most common words', count[4:10])
print('Sample data', data[:10], [rev_dictionary[i] for i in data[:10]])

vocab from size: 20332
Most common words [('film', 1453), ('movie', 1270), ('one', 727), ('like', 721), ('story', 477), ('much', 386)]
Sample data [2417, 170, 1205, 92, 401, 87, 950, 1517, 230, 5347] ['alternately', 'fascinating', 'frustrating', 'documentary', 'psychological', 'thriller', 'genuinely', 'spooky', 'premise', 'aboveaverage']


In [7]:
GO = dictionary['GO']
PAD = dictionary['PAD']
EOS = dictionary['EOS']
UNK = dictionary['UNK']

In [25]:
class Model:
    def __init__(self, size_layer, num_layers, embedded_size, dict_size, dimension_output, learning_rate, attention_size=150):
        
        def cells(reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size_layer,initializer=tf.orthogonal_initializer(),reuse=reuse)
        
        # (B, S) batch_size/seq_len
        self.X = tf.placeholder(tf.int32, [None, None])
        # (?, 2)
        self.Y = tf.placeholder(tf.float32, [None, dimension_output])
        # (20336, 128)
        encoder_embeddings = tf.Variable(tf.random_uniform([dict_size, embedded_size], -1, 1))
        # (B, S, 128)
        encoder_embedded = tf.nn.embedding_lookup(encoder_embeddings, self.X)
        # 2层LSTM 每层128个神经元
        rnn_cells = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)])
        # outputs: (B, S, 128) last_state: c/h (B, 128)*2
        outputs, last_state = tf.nn.dynamic_rnn(rnn_cells, encoder_embedded, dtype = tf.float32)
        # (150,)
        attention_w = tf.get_variable("attention_v", [attention_size], tf.float32)
        # last_state[-1].h: (B, 128)    tf.expand_dims(last_state[-1].h, 1): (B, 1, 128)    query:  (B, 1, 150)    
        query = tf.layers.dense(tf.expand_dims(last_state[-1].h, 1), attention_size)
        # (B, S, 150)
        keys = tf.layers.dense(outputs, attention_size)
        # keys + query: (B, S, 150)     tf.tanh(keys + query): (B, S, 150)     attention_w * tf.tanh(keys + query): 
        # (B, S, 150)  align: (B, S)
        align = tf.reduce_sum(attention_w * tf.tanh(keys + query), [2])
        # (B, S)
        align = tf.nn.softmax(align)    
        # tf.transpose(outputs, [0, 2, 1]): (?, 128, ?)     tf.expand_dims(align, 2): (?, ?, 1)
        # tf.matmul(tf.transpose(outputs, [0, 2, 1]), tf.expand_dims(align, 2)): (?, 128, 1)
        # outputs: (?, 128)
        outputs = tf.squeeze(tf.matmul(tf.transpose(outputs, [0, 2, 1]), tf.expand_dims(align, 2)), 2)
        # size_layer: 128   dimension_output: 2
        W = tf.get_variable('w',shape=(size_layer, dimension_output),initializer=tf.orthogonal_initializer())
        # dimension_output: 2
        b = tf.get_variable('b',shape=(dimension_output),initializer=tf.zeros_initializer())
        # outputs: (?, 128)     W: (128, 2)     b: (2,)
        self.logits = tf.matmul(outputs, W) + b
        # Y: (?, 2)
        self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = self.logits, labels = self.Y))
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        correct_pred = tf.equal(tf.argmax(self.logits, 1), tf.argmax(self.Y, 1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

size_layer = 128
num_layers = 2
embedded_size = 128
dimension_output = len(trainset.target_names)
learning_rate = 1e-3
maxlen = 50
batch_size = 128


tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model(size_layer,num_layers,embedded_size,vocabulary_size+4,dimension_output,learning_rate)
sess.run(tf.global_variables_initializer())


EARLY_STOPPING, CURRENT_CHECKPOINT, CURRENT_ACC, EPOCH = 5, 0, 0, 0
while True:
    lasttime = time.time()
    if CURRENT_CHECKPOINT == EARLY_STOPPING:
        print('break epoch:%d\n'%(EPOCH))
        break
        
    train_acc, train_loss, test_acc, test_loss = 0, 0, 0, 0
    for i in range(0, (len(train_X) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(train_X[i:i+batch_size],dictionary,maxlen)
        acc, loss, _ = sess.run([model.accuracy, model.cost, model.optimizer], 
                           feed_dict = {model.X : batch_x, model.Y : train_onehot[i:i+batch_size]})
        train_loss += loss
        train_acc += acc
    
    for i in range(0, (len(test_X) // batch_size) * batch_size, batch_size):
        batch_x = str_idx(test_X[i:i+batch_size],dictionary,maxlen)
        acc, loss = sess.run([model.accuracy, model.cost], 
                           feed_dict = {model.X : batch_x, model.Y : test_onehot[i:i+batch_size]})
        test_loss += loss
        test_acc += acc
    
    train_loss /= (len(train_X) // batch_size)
    train_acc /= (len(train_X) // batch_size)
    test_loss /= (len(test_X) // batch_size)
    test_acc /= (len(test_X) // batch_size)
    
    if test_acc > CURRENT_ACC:
        print('epoch: %d, pass acc: %f, current acc: %f'%(EPOCH,CURRENT_ACC, test_acc))
        CURRENT_ACC = test_acc
        CURRENT_CHECKPOINT = 0
    else:
        CURRENT_CHECKPOINT += 1
        
    print('time taken:', time.time()-lasttime)
    print('epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'%(EPOCH,train_loss,
                                                                                          train_acc,test_loss,
                                                                                          test_acc))
    EPOCH += 1

KeyboardInterrupt: 

In [12]:
logits = sess.run(model.logits, feed_dict={model.X:str_idx(test_X,dictionary,maxlen)})
print(metrics.classification_report(test_Y, np.argmax(logits,1), target_names = trainset.target_names))

              precision    recall  f1-score   support

    negative       0.69      0.78      0.73      1082
    positive       0.74      0.64      0.69      1051

    accuracy                           0.71      2133
   macro avg       0.71      0.71      0.71      2133
weighted avg       0.71      0.71      0.71      2133

